In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import glob, os, psutil, time

# c) simulated data

In [ ]:

data_path = '../fits/ERC/nonlinear_fitzhughnagumo/'
fig_path = '../figs/'

pars=np.load(data_path+'pars_true.npz')['pars'].tolist()
pars_est_full   = np.load(data_path+'pars_est_full.npz')['arr_0'].tolist()['pars_est']
pars_est_partial= np.load(data_path+'pars_est_partial_cont.npz')['arr_0'].tolist()['pars_est']
pars_pca        = np.load(data_path+'pars_pca.npz')['pars'].tolist()
#pars_est_pca= np.load(data_path+'pars_est_pca.npz')['pars'].tolist()

lag_range = range(0,21,4)

idx_a = pars['idx_a']
idx_b = pars['idx_b']
pa, pb = idx_a.size, idx_b.size
x=np.load(data_path+'x.npz')['x']

p,n = pars['C'].shape
T = x.shape[0]
k,l = pars['k'],pars['l']

mmap, chunksize = True, np.min((T,p))
        
y = np.memmap(data_path+'y', dtype=np.float, mode='r', shape=(T,p))

# c) nonlinear dynamics + linear emissions, latent and observed traces

In [ ]:
tmin, tmax = 1000, 1500

idx_showy = np.arange(15)
yshow = y[tmin:tmax,idx_showy]
ydiff = yshow.max() - yshow.min()

idx_showx = np.array([0,10,1,11,2,12])
xshow = x[tmin:tmax,idx_showx]
xdiff = xshow.max() - xshow.min()

plt.figure(figsize=(2.5,2.5))
ax = plt.subplot(111)
for i in range(idx_showy.size):
    plt.plot(range(tmin,tmax), i*ydiff + yshow[:,i], 'k')
    plt.hold(True)
    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.xlim(tmin, tmax)    
plt.xticks(range(tmin, tmax+1, 500), [str(x) for x in range(tmin, tmax+1, 500)], fontsize=14)    

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    
plt.yticks([])    

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    
plt.xlim(tmin, tmax)    

plt.tick_params(axis="both", which="both", top="off", left='off', right="off", labelleft="on")    
plt.box('off')

plt.savefig(fig_path + 'c_simulated_x.pdf', bbox_inches="tight"); 
plt.show()



plt.figure(figsize=(2.5,2.5))
ax = plt.subplot(111)

for i in range(idx_showx.size):
    plt.plot(range(tmin,tmax), -(xdiff+ydiff)-i*xdiff + xshow[:,i], color=[.7,.7,.7])
    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.xlim(tmin, tmax)    
plt.xticks(range(tmin, tmax+1, 500), [str(x) for x in range(tmin, tmax+1, 500)], fontsize=14)    

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    
plt.yticks([])    
# Remove the tick marks
plt.tick_params(axis="both", which="both", top="off", left='off', right="off", labelleft="on")    
plt.box('off')
plt.savefig(fig_path + 'c_simulated_z.pdf', bbox_inches="tight"); 
plt.show()


# c) quality of fit (snippets of covariance matrices)

In [ ]:
sub_pops = (np.arange(5500), np.arange(4500,p))

nshow = 10
idx_a_show = np.setdiff1d(np.arange(p), sub_pops[1])
idx_b_show = np.setdiff1d(np.arange(p), sub_pops[0])

idx_a_show = idx_a_show[np.random.choice(len(idx_a_show), nshow, replace=False)]
idx_b_show = idx_b_show[np.random.choice(len(idx_b_show), nshow, replace=False)]

for m in range(1):
    print('computing time-lagged covariance for lag ', str(m))

    Qr = np.cov(y[m:m-(k+l),idx_a_show].T, y[:-(k+l),idx_b_show].T)[:idx_a_show.size,idx_b_show.size:]
    Qf = (pars_est_full['C'][idx_a_show,:].dot(pars_est_full['X'][:n,:]).dot(pars_est_full['C'][idx_b_show,:].T)) + np.diag(pars_est_full['R'])[np.ix_(idx_a_show,idx_b_show)]
    Qp = (pars_est_partial['C'][idx_a_show,:].dot(pars_est_partial['X'][:n,:]).dot(pars_est_partial['C'][idx_b_show,:].T)) + np.diag(pars_est_full['R'])[np.ix_(idx_a_show,idx_b_show)]

    cm = np.min((Qr.min(), Qf.min(), Qp.min()))
    cM = np.max((Qr.max(), Qf.max(), Qp.max()))
    
    plt.figure(figsize=(3, 3))
    plt.imshow(Qr, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('true')
    plt.savefig(fig_path + 'c_simulated_true_cov.pdf', bbox_inches="tight");     
    plt.show()
    

    plt.figure(figsize=(3, 3))
    plt.imshow(Qf, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('full')
    plt.savefig(fig_path + 'c_simulated_full_cov.pdf', bbox_inches="tight");     
    plt.show()
    

    plt.figure(figsize=(3, 3))
    plt.imshow(Qp, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('stitch')
    plt.savefig(fig_path + 'c_simulated_stitch_cov.pdf', bbox_inches="tight");     
    plt.show()


# c) combining fully / partially obs traces tau vs. corr

In [ ]:
sub_pops = (np.arange(5500), np.arange(4500,p))
idx_a = np.setdiff1d(np.arange(p), sub_pops[1])
idx_b = np.setdiff1d(np.arange(p), sub_pops[0])
pa, pb = idx_a.size, idx_b.size


corrs_true, corrs_est_full, corrs_est_partial = np.zeros(len(lag_range)), np.zeros(len(lag_range)), np.zeros(len(lag_range))
corrs_est_pca = np.zeros(len(lag_range))
kl_ = np.max(lag_range)+1
for m in range(len(lag_range)):
    m_ = lag_range[m]
    print('computing time-lagged covariance for lag ', str(m))
    
    Qf= np.cov(y[m_:m_-kl_,idx_a].T, y[:-kl_,idx_b].T)[:pa,pa:]   

    Qr = pars['C'][idx_a,:].dot(np.cov(x[m_:m_-kl_,:].T, x[0:-kl_,:].T)[:n,n:]).dot(pars['C'][idx_b,:].T)
    Qr = Qr + np.diag(pars['R'])[idx_a,idx_b] if m == 0 else Qr
    
    corrs_true[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]
    
    Qr = pars_est_full['C'][idx_a,:].dot(pars_est_full['X'][m*n:(m+1)*n,:]).dot(pars_est_full['C'][idx_b,:].T)

    corrs_est_full[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]

    Qr = pars_est_partial['C'][idx_a,:].dot(pars_est_partial['X'][m*n:(m+1)*n,:]).dot(pars_est_partial['C'][idx_b,:].T)

    corrs_est_partial[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]
    
    Qr = pars_pca['C'][idx_a,:].dot(pars_pca['Pi']).dot(pars_pca['C'][idx_b,:].T)
    corrs_est_pca[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]
   

# (comparison full vs. partial)

In [ ]:
plt.figure(figsize=(3, 2))
taus = lag_range
plt.plot(taus, corrs_est_full, 'd-', color='b')
#plt.plot(taus, corrs_true, 'o-', color='k')
plt.hold(True)
plt.plot(taus, corrs_est_partial, 'd-', color='r')
#plt.plot(taus, corrs_est_pca, 'd-', color='k')
plt.axis([-0.5, np.max(taus)+0.5, 0.8, 1])
plt.legend(('fully obs. fit', 'part. obs. fit', 'part. obs. PCA'), 'lower left')
plt.legend
plt.xlabel('time-lag [frames]')
plt.ylabel('corr(\cov(y_{t+tau}, y_t), C X_tau C'')')

ax = plt.subplot(111)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.yticks(np.arange(0.80, 1.01, 0.05), [str(x) for x in np.arange(0.80, 1.01, 0.05)], fontsize=14)    
plt.xticks(taus, [str(x) for x in taus], fontsize=14)


ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on")    
#plt.box('off')

plt.savefig(fig_path + 'c_simulated_performance.pdf', bbox_inches="tight"); 
plt.show()

# (comparison full vs. partial vs. partial PCA vs. ground-truth)

In [ ]:
plt.figure(figsize=(6, 4))
taus = lag_range
plt.plot(taus, corrs_true, 'd-', color='g')
plt.plot(taus, corrs_est_full, 'd-', color='b')
plt.hold(True)
plt.plot(taus, corrs_est_partial, 'd-', color='r')
plt.plot(taus, corrs_est_pca, 'd-', color='k')


plt.legend(('ground-truth par.','fully obs. fit', 'part. obs. fit', 'part. obs. PCA'), 'lower left')
plt.legend
plt.xlabel('time-lag [frames]')
plt.ylabel('corr(\cov(y_{t+tau}, y_t), C X_tau C'')')

ax = plt.subplot(111)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.axis([-0.5, np.max(taus)+0.5, -0.3, 1.01])
plt.yticks(np.arange(-0.2, 1.01, 0.25), [str(x) for x in np.arange(-0.20, 1.01, 0.25)], fontsize=14)    
plt.xticks(taus, [str(x) for x in taus], fontsize=14)
  

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on")    
#plt.box('off')

plt.savefig(fig_path + 'c_simulated_performance_pca_grtr.pdf', bbox_inches="tight"); 
plt.show()









# d) real data

In [ ]:

data_path = '../fits/lsfm/grid_quick/'
fig_path = '../figs/'

tmp = np.load(data_path + 'stitch_2pops_zsplit_6_lags.npz')
p,n,T = tmp['p'],tmp['n'],tmp['T']
lag_range = tmp['lag_range']
pars_est_partial = tmp['pars_est'].tolist()
sub_pops = tmp['sub_pops']
obs_pops = np.arange(len(sub_pops))
obs_time = T//len(sub_pops) * np.arange(1,len(sub_pops)+1)
    
tmp = np.load(data_path + 'full_2pops_zsplit_6_lags.npz')
pars_est_full = tmp['pars_est'].tolist()
y = np.memmap(data_path+'y_zscore', dtype=np.float, mode='r', shape=(T,p))



In [ ]:
nz, nx, ny = 41, 1024, 2048
filename = '/home/mackelab/data/dOMR0_20150414_112406/TM00000_CM0_CHN00.stack'
stack = np.fromfile(filename, dtype=np.float16).reshape(nz, nx, ny).astype(dtype=np.float32)

def sig(x):
    #return 1/(1+np.exp(-x/2.))
    return x

for z in range(nz):
    plt.figure(figsize=(4,5))
    plt.imshow( sig((stack[z,:,:].T-stack[z,:,:].mean())/np.sqrt(np.var(stack[z,:,:]))), interpolation='None')
    plt.gray()
    plt.title('z =' + str(z), fontsize=14)
    plt.yticks([0, 1024, 2048], [str(x) for x in [0, 1024, 2048]], fontsize=14)
    plt.xticks([0, 1024], [str(x) for x in [0, 1024]], fontsize=14)
    plt.savefig(fig_path + 'd_frame0_raw_z'+ str(z+1) +'.pdf', bbox_inches="tight");     
    plt.show()

# d) quality of fit (snippets of covariance matrices)

In [ ]:
nshow = 10
idx_a_show = np.setdiff1d(np.arange(p), sub_pops[1])
idx_b_show = np.setdiff1d(np.arange(p), sub_pops[0])

idx_a_show = idx_a_show[np.random.choice(len(idx_a_show), nshow, replace=False)]
idx_b_show = idx_b_show[np.random.choice(len(idx_b_show), nshow, replace=False)]


kl_ = np.max(lag_range)+1
for m in range(1):
    print('computing time-lagged covariance for lag ', str(m))
    m_ = lag_range[m]
    
    Qr = np.cov(y[m_:m_-kl_,idx_a_show].T, y[:-kl_,idx_b_show].T)[:idx_a_show.size,idx_b_show.size:]
    Qf = (pars_est_full['C'][idx_a_show,:].dot(pars_est_full['X'][:n,:]).dot(pars_est_full['C'][idx_b_show,:].T)) + np.diag(pars_est_full['R'])[np.ix_(idx_a_show,idx_b_show)]
    Qp = (pars_est_partial['C'][idx_a_show,:].dot(pars_est_partial['X'][:n,:]).dot(pars_est_partial['C'][idx_b_show,:].T)) + np.diag(pars_est_full['R'])[np.ix_(idx_a_show,idx_b_show)]

    cm = np.min((Qr.min(), Qf.min(), Qp.min()))
    cM = np.max((Qr.max(), Qf.max(), Qp.max()))
    
    plt.figure(figsize=(3, 3))
    plt.imshow(Qr, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('true')
    plt.savefig(fig_path + 'd_lsfm_true_cov.pdf', bbox_inches="tight");     
    plt.show()
    

    plt.figure(figsize=(3, 3))
    plt.imshow(Qf, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('full')
    plt.savefig(fig_path + 'd_lsfm_full_cov.pdf', bbox_inches="tight");     
    plt.show()
    

    plt.figure(figsize=(3, 3))
    plt.imshow(Qp, interpolation = 'None')
    plt.colorbar()
    plt.clim(cm, cM)
    plt.gray()
    plt.title('stitch')
    plt.savefig(fig_path + 'd_lsfm_stitch_cov.pdf', bbox_inches="tight");     
    plt.savefig(fig_path + 'd_lsfm_stitch_cov.pdf', bbox_inches="tight");     
    plt.show()


# d) combining fully / partially obs traces tau vs. corr

In [ ]:
from sklearn.decomposition import PCA
y_masked = np.memmap(data_path+'y_zscore', dtype=np.float, mode='c', shape=(T,p))
y_masked[:obs_time[0], np.setdiff1d(np.arange(p), sub_pops[obs_pops[0]])] = 0
for i in range(1,len(obs_time)):
    y_masked[np.ix_(np.arange(obs_time[i-1],obs_time[i]), np.setdiff1d(np.arange(p), sub_pops[obs_pops[i]]))] = 0
    
pca = PCA()
pca.fit(y_masked)
del y_masked
pars_pca = {}
pars_pca['C'] = pca.components_[:n].T
pars_pca['Pi'] = np.diag(pca.explained_variance_[:n])    

print('done PCA')

In [ ]:
idx_a = np.setdiff1d(np.arange(p), sub_pops[1])
idx_b = np.setdiff1d(np.arange(p), sub_pops[0])
pa, pb = idx_a.size, idx_b.size


corrs_est_full, corrs_est_partial = np.zeros(len(lag_range)), np.zeros(len(lag_range))
corrs_est_pca = np.zeros(len(lag_range))
kl_ = np.max(lag_range)+1
for m in range(len(lag_range)):
    m_ = lag_range[m]
    print('computing time-lagged covariance for lag ', str(m_))
    Q = np.cov(y[m_:m_-kl_,idx_a].T, y[:-kl_,idx_b].T)[:pa,pa:]     
    Qf= np.cov(y[m_:m_-kl_,idx_a].T, y[:-kl_,idx_b].T)[:pa,pa:]   
    
    Qr = pars_est_full['C'][idx_a,:].dot(pars_est_full['X'][m*n:(m+1)*n,:]).dot(pars_est_full['C'][idx_b,:].T)

    corrs_est_full[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]

    Qr = pars_est_partial['C'][idx_a,:].dot(pars_est_partial['X'][m*n:(m+1)*n,:]).dot(pars_est_partial['C'][idx_b,:].T)

    corrs_est_partial[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]
    
    Qr = pars_pca['C'][idx_a,:].dot(pars_pca['Pi']).dot(pars_pca['C'][idx_b,:].T)
    corrs_est_pca[m] = np.corrcoef(Qf.reshape(-1), Qr.reshape(-1))[0,1]


In [ ]:
np.diag(pars_pca['Pi'])

# (comparison full vs. partial)

In [ ]:
plt.figure(figsize=(3, 2))
taus = lag_range
plt.plot(taus, corrs_est_full, 'd-', color='b')
#plt.plot(taus, corrs_true, 'o-', color='k')
plt.hold(True)
plt.plot(taus, corrs_est_partial, 'd-', color='r')

plt.axis([-0.5, np.max(taus)+0.5, 0.45, 0.67])

plt.legend(('fully obs. fit', 'part. obs. fit'), 'upper right')
plt.legend
plt.xlabel('time-lag [frames]')
plt.ylabel('corr(\cov(y_{t+tau}, y_t), C X_tau C'')')

ax = plt.subplot(111)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.yticks(np.arange(0.45, 0.66, 0.1), [str(x) for x in np.arange(0.45, 0.66, 0.1)], fontsize=14)    
plt.xticks(taus, [str(x) for x in taus], fontsize=14)

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on")    
#plt.box('off')

plt.savefig(fig_path + 'd_real_performance.pdf', bbox_inches="tight"); 
plt.show()

# (comparison full vs. partial vs. partial PCA vs. ground-truth)

In [ ]:
plt.figure(figsize=(3, 2))
taus = lag_range
plt.plot(taus, corrs_est_full, 'd-', color='b')
#plt.plot(taus, corrs_true, 'o-', color='k')
plt.hold(True)
plt.plot(taus, corrs_est_partial, 'd-', color='r')
plt.plot(taus, corrs_est_pca, 'd-', color='k')

plt.axis([-0.5, np.max(taus)+0.5, 0.45, 0.71])

plt.legend(('fully obs. fit', 'part. obs. fit', 'part. obs. PCA'), 'upper right')
plt.legend
plt.xlabel('time-lag [frames]')
plt.ylabel('corr(\cov(y_{t+tau}, y_t), C X_tau C'')')

ax = plt.subplot(111)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.yticks(np.arange(0.4, 0.71, 0.1), [str(x) for x in np.arange(0.4, 0.71, 0.1)], fontsize=14)    
plt.xticks(taus, [str(x) for x in taus], fontsize=14)
plt.xticks(fontsize=10)    

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on")    
#plt.box('off')

plt.savefig(fig_path + 'd_real_performance_pca.pdf', bbox_inches="tight"); 
plt.show()